# Parse data from Google Sheet for tab "normalized-cms"

## Doing
 - [ ] Merge Provider info with IL Facility CSV 
## To Do's
 - [ ] Check Normalized names in spreadsheet, seems not all names Normailized ex: "ACCOLADE HEALTHCARE" and "Accolade Healthcare"
 - [ ] Print list of names that would match if Normailized - Try for each matched Fed ID - check Normalized Names, Facility Name and do lookup to see if matching Facility Names with different Facility Normailized
 - [ ] Lookup from CMS data feed as opposed to the spreadsheet VLOOKUPs
 - [ ] Rename columns to prefix with data source: CTP_LTC_ and CMS_
 - [ ] define and add data quality checks on feeds (ex dup checks if not allowed, etc)
 - [ ] Compare CMS stats just b4 Covid and now
 
## Done
 - [x] Get Counts, Sums and Means by Ownership type and State to see if any State way better/worse based on Ownership Type
 - [x] Get Counts, Sums and Means by Ownership type
 - [x] Join CMS data to State LTC data into DF for Provider Info
 - [x] Create Facility Name to Provider Number for State to CMS mapping.
 - [x] Add "Facility Normalize Name" - which is the name used to match in CMS feed to get Federal Provider Number
 - [x] Check for more than one "Normalized Name" for the same "Facility Name"

In [1]:
from gsheets import Sheets

import json
import pandas as pd
pd.options.display.max_columns = None

# Define Functions

In [2]:
def parse_noarmalized_cms_sheet (df):
    """ # Create DFs for:
        #   1 - no_matches: Lists of "Factility Names" and "Facility Names Normalized" that do not have a matching CMS entry in the normalized-cms sheet
        #   2 - dupNames: Dictionary of "Federal Provider Numbers" that map to list where there are more than one "Facility Name"
        #   3 - CMSID2FacName: Dictionary mapping "Facility Names" to "Federal Provider Numbers"
        #   4 - FacName2CMSId: Dictionary mapping "Federal Provider Numbers" to "Facility Names"
    """
    FacName2CMSId = {}
    CMSID2FacName = {} # Initialize No Match to empty list so can always use extend for this key
    no_matches = {'Facility Names': [],
                  'Facility Names Normalized': []} # List of "Facility Names" without matches in CMS data feed
    dupNames = {} # Dictionary of "Federal Provider Numbers" that have more than one "Facility Name" 
                  #  (ex: 146182': ['Alden Courts of Waterford', 'Alden of Waterford'],
                  #      '146186': ['Alden Estates Courts of Huntley',
                  #                 'Alden Estates-Courts of Huntley'],
                  #       '145872': ['Alden Long Grove Rehab & Health',
                  #                  'Alden of Long Grove Rehab and HCC'], )

    # For Each Federal Provder Number (CMS term) in Provide ID - CMS (CTP term)
    for Fed_Prov_Num in df['Provider ID - CMS'].unique():
        # Since the gsheets api appends comments to the cell value (which for the record can be a PAIN and POTENTIAL PROBLEM)
        #    We change the value to a more understandable "No Match"
        if str(Fed_Prov_Num)[0:4]== "#N/A":
            no_matches['Facility Names'].extend(list(df[df['Provider ID - CMS'] == Fed_Prov_Num]['Facility Name'].unique()))
            no_matches['Facility Names Normalized'].extend(list(df[df['Provider ID - CMS'] == Fed_Prov_Num]['Facility Name Normalized'].unique()))
        else:
            CMSID2FacName[Fed_Prov_Num] = list(df[df['Provider ID - CMS'] == Fed_Prov_Num]['county-facName'].dropna().unique())
            if len(CMSID2FacName[Fed_Prov_Num]) > 1:
                dupNames[Fed_Prov_Num] = CMSID2FacName[Fed_Prov_Num]

            for facilityName in CMSID2FacName[Fed_Prov_Num]:
                FacName2CMSId[facilityName] = Fed_Prov_Num

    no_matches['Facility Names'] = list(dict.fromkeys(no_matches['Facility Names']))
    no_matches['Facility Names Normalized'] = list(dict.fromkeys(no_matches['Facility Names Normalized']))
    return FacName2CMSId, CMSID2FacName, no_matches, dupNames

# Get Info From SpreadSheet

In [3]:
sheets = Sheets.from_files('credentials.json', '~/storage.json')

In [4]:
url = 'https://docs.google.com/spreadsheets/d/14V12Hmx5B3BXsEBklYzDVC3mbnRDPSqJdsXHdzmO6yI'
s = sheets.get(url)

In [17]:
ns_sheet = s.find('normalized-state')
norm_state_df = ns_sheet.to_frame()
norm_state_df = norm_state_df.assign(FacilityNameNormalizedUCase = lambda x: (x['Facility Name'].str.upper()))
norm_state_df['county-facName']= norm_state_df['County'].str.upper() + '-' +norm_state_df['Facility Name'].str.upper()
[FacName2CMSId, CMSID2FacName, no_matches, dupNames] = parse_noarmalized_cms_sheet(norm_state_df)

with open("IL_FacilityName_to_CMS_ID.json", "w") as outfile:  
    json.dump(FacName2CMSId, outfile) 

In [18]:
# Get Quick Stats on number of names
print("Unique Facility Name count: " + str(len(norm_state_df['Facility Name'].unique())))
print("Unique Facility Name UPPER count: " + str(len(norm_state_df['Facility Name'].str.upper().unique())))
print("Unique Facility Normalized count: " + str(len(norm_state_df['Facility Name Normalized'].unique())))
print("Unique Facility Normalized UPPER count: " + str(len(norm_state_df['Facility Name Normalized'].str.upper().unique())))
print("Facilites with no match in CMS: ", len(no_matches['Facility Names']))
print("Number of CMS entries matched to Federal Provider Number: ", len(CMSID2FacName))
print('"Federal Provider Numbers" that have more than one "Facility Name": ', len(dupNames))

Unique Facility Name count: 1214
Unique Facility Name UPPER count: 1201
Unique Facility Normalized count: 2077
Unique Facility Normalized UPPER count: 1282
Facilites with no match in CMS:  726
Number of CMS entries matched to Federal Provider Number:  541
"Federal Provider Numbers" that have more than one "Facility Name":  26


# Load CMS data

In [19]:
# added engine='python' because c engine does not support skipfooter
#cms_df = pd.read_csv('../CMS NursingHome Data/2013-07-01/ProviderInfo.csv', skipfooter=1, engine='python')
cms_df = pd.read_csv('../CMS NursingHome Data/NH_ProviderInfo_2020-10-30.csv', skipfooter=1, engine='python')
cms_df

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Average Number of Residents per Day Footnote,Provider Type,Provider Resides in Hospital,Legal Business Name,Date First Approved to Provide Medicare and Medicaid Services,Continuing Care Retirement Community,Special Focus Status,Abuse Icon,Most Recent Health Inspection More Than 2 Years Ago,Provider Changed Ownership in Last 12 Months,With a Resident and Family Council,Automatic Sprinkler Systems in All Required Areas,Overall Rating,Overall Rating Footnote,Health Inspection Rating,Health Inspection Rating Footnote,QM Rating,QM Rating Footnote,Long-Stay QM Rating,Long-Stay QM Rating Footnote,Short-Stay QM Rating,Short-Stay QM Rating Footnote,Staffing Rating,Staffing Rating Footnote,RN Staffing Rating,RN Staffing Rating Footnote,Reported Staffing Footnote,Physical Therapist Staffing Footnote,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Case-Mix Nurse Aide Staffing Hours per Resident per Day,Case-Mix LPN Staffing Hours per Resident per Day,Case-Mix RN Staffing Hours per Resident per Day,Case-Mix Total Nurse Staffing Hours per Resident per Day,Adjusted Nurse Aide Staffing Hours per Resident per Day,Adjusted LPN Staffing Hours per Resident per Day,Adjusted RN Staffing Hours per Resident per Day,Adjusted Total Nurse Staffing Hours per Resident per Day,Rating Cycle 1 Standard Survey Health Date,Rating Cycle 1 Total Number of Health Deficiencies,Rating Cycle 1 Number of Standard Health Deficiencies,Rating Cycle 1 Number of Complaint Health Deficiencies,Rating Cycle 1 Health Deficiency Score,Rating Cycle 1 Number of Health Revisits,Rating Cycle 1 Health Revisit Score,Rating Cycle 1 Total Health Score,Rating Cycle 2 Standard Health Survey Date,Rating Cycle 2 Total Number of Health Deficiencies,Rating Cycle 2 Number of Standard Health Deficiencies,Rating Cycle 2 Number of Complaint Health Deficiencies,Rating Cycle 2 Health Deficiency Score,Rating Cycle 2 Number of Health Revisits,Rating Cycle 2 Health Revisit Score,Rating Cycle 2 Total Health Score,Rating Cycle 3 Standard Health Survey Date,Rating Cycle 3 Total Number of Health Deficiencies,Rating Cycle 3 Number of Standard Health Deficiencies,Rating Cycle 3 Number of Complaint Health Deficiencies,Rating Cycle 3 Health Deficiency Score,Rating Cycle 3 Number of Health Revisits,Rating Cycle 3 Health Revisit Score,Rating Cycle 3 Total Health Score,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,290,Franklin,For profit - Corporation,57,44.9,NaN,Medicare and Medicaid,N,"BURNS NURSING HOME, INC.",1969-09-01,N,NaN,N,N,N,Both,Yes,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,2.73869,0.96561,0.91573,1.88134,4.62003,0.00000,2.16170,0.70657,0.32721,3.19548,2.61545,1.00799,1.08075,4.60948,2019-08-21,2,2,0,8,1,0,8,2018-08-01,1,1,0,4,1,0,4,2017-06-22,0,0,0,0,0,0,0,5.333,0,0,0,0,0,0,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",2020-10-01
1,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,For profit - Corporation,85,76.1,NaN,Medicare and Medicaid,N,COOSA VALLEY HEALTHCARE CENTER LLC,1967-01-01,N,NaN,N,N,N,Both,Yes,4.0,NaN,3.0,NaN,4.0,NaN,2.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,3.04618,0.96901,0.89255,1.86156,4.90774,0.01443,1.99014,0.69526,0

In [20]:
merged_df = pd.merge(norm_state_df, cms_df, left_on='Provider ID - CMS',  right_on='Federal Provider Number', how='left')
merged_df

,Facility Name,Facility Name Normalized,VLOOKUP,Provider ID - CMS,Address - CMS,City - CMS,Overall Star Ratings CMS,Health Inspection Rating_x,QM Rating_x,Staffing Rating_x,Owner type,Owner name,Organization Type,Provider Type_x,Legal Business Name_x,Changes,Date Collected,State,County,City,State Facility Type,CTP Facility Categorization,State/Fed Regulated,State Facility ID,CMS Facility ID,Date outbreak opened,Date outreak closed,Outbreak Status,Resident Census,Resident Positives,Resident Probable Positives,Resident Deaths,Resident Probable Deaths,Staff Positive,Staff Probable Positives,Staff Deaths,Staff Probable Deaths,Resident/Staff Positives,Resident/Staff Probable Positives,Resident/Staff Deaths,Resident/Staff Probable Deaths,Resident Positives.1,Resident Probable Positives.1,Resident Deaths.1,Resident Probable Deaths.1,Staff Positive.1,Staff Probable Positives.1,Staff Deaths.1,Staff Probable Deaths.1,Resident/Staff Positives.1,Resident/Staff Probable Positives.1,Resident/Staff Deaths.1,Resident/Staff Probable Deaths.1,Residents Tested,Staff Tested,Personal Protective Equipment,FacilityNameNormalizedUCase,county-facName,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Average Number of Residents per Day Footnote,Provider Type_y,Provider Resides in Hospital,Legal Business Name_y,Date First Approved to Provide Medicare and Medicaid Services,Continuing Care Retirement Community,Special Focus Status,Abuse Icon,Most Recent Health Inspection More Than 2 Years Ago,Provider Changed Ownership in Last 12 Months,With a Resident and Family Council,Automatic Sprinkler Systems in All Required Areas,Overall Rating,Overall Rating Footnote,Health Inspection Rating_y,Health Inspection Rating Footnote,QM Rating_y,QM Rating Footnote,Long-Stay QM Rating,Long-Stay QM Rating Footnote,Short-Stay QM Rating,Short-Stay QM Rating Footnote,Staffing Rating_y,Staffing Rating Footnote,RN Staffing Rating,RN Staffing Rating Footnote,Reported Staffing Footnote,Physical Therapist Staffing Footnote,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Case-Mix Nurse Aide Staffing Hours per Resident per Day,Case-Mix LPN Staffing Hours per Resident per Day,Case-Mix RN Staffing Hours per Resident per Day,Case-Mix Total Nurse Staffing Hours per Resident per Day,Adjusted Nurse Aide Staffing Hours per Resident per Day,Adjusted LPN Staffing Hours per Resident per Day,Adjusted RN Staffing Hours per Resident per Day,Adjusted Total Nurse Staffing Hours per Resident per Day,Rating Cycle 1 Standard Survey Health Date,Rating Cycle 1 Total Number of Health Deficiencies,Rating Cycle 1 Number of Standard Health Deficiencies,Rating Cycle 1 Number of Complaint Health Deficiencies,Rating Cycle 1 Health Deficiency Score,Rating Cycle 1 Number of Health Revisits,Rating Cycle 1 Health Revisit Score,Rating Cycle 1 Total Health Score,Rating Cycle 2 Standard Health Survey Date,Rating Cycle 2 Total Number of Health Deficiencies,Rating Cycle 2 Number of Standard Health Deficiencies,Rating Cycle 2 Number of Complaint Health Deficiencies,Rating Cycle 2 Health Deficiency Score,Rating Cycle 2 Number of Health Revisits,Rating Cycle 2 Health Revisit Score,Rating Cycle 2 Total Health Score,Rating Cycle 3 Standard Health Survey Date,Rating Cycle 3 Total Number of Health Deficiencies,Rating Cycle 3 Number of Standard Health Deficiencies,Rating Cycle 3 Number of Complaint Health Deficiencies,Rating Cycle 3 Health Deficiency Score,Rating Cycle 3 Number of Health Revisits,Rating Cycle 3 Health Revisit Score,Rating Cycle 3 Total Health

In [21]:
cols = ['PROVNUM', 'PROVNAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE',
        'COUNTY_SSA', 'County_name', 'OWNERSHIP', 'BEDCERT', 'RESTOT', 'CERTIFICATION',
        'LBN',  'Overall_Rating', 'overall_rating_fn', 'INHOSP', 'INCIDENT_CNT', 'CMPLNT_CNT', 'FINE_CNT',
        'FINE_TOT', 'PAYDEN_CNT', 'TOT_PENLTY_CNT', 'FILEDATE']
        
#        'restot_fn', 'INHOSP', 'PARTICIPATION_DATE',
#        'CCRC_FACIL', 'SFFStatus', 'ABUSE_ICON', 'OldSurvey', 'CHOW_LAST_12MOS',
#        'RESFAMCOUNCIL', 'SPRINKLER_STATUS', 'Overall_Rating',
#        'overall_rating_fn', 'SURVEY_RATING', 'survey_rating_fn',
#        'Quality_Rating', 'quality_rating_fn', 'LS_Quality_Rating',
#        'LS_quality_rating_fn', 'SS_Quality_Rating', 'SS_quality_rating_fn',
#        'Staffing_Rating', 'staffing_rating_fn', 'RN_staffing_rating',
#        'RN_staffing_rating_fn', 'STAFFING_FLAG', 'PT_STAFFING_FLAG', 'AIDHRD',
#        'VOCHRD', 'RNHRD', 'TOTLICHRD', 'TOTHRD', 'PTHRD', 'CM_AIDE', 'CM_LPN',
#        'CM_RN', 'CM_TOTAL', 'ADJ_AIDE', 'ADJ_LPN', 'ADJ_RN', 'ADJ_TOTAL',
#        'WEIGHTED_ALL_CYCLES_SCORE', 'INCIDENT_CNT', 'CMPLNT_CNT', 'FINE_CNT',
#        'FINE_TOT', 'PAYDEN_CNT', 'TOT_PENLTY_CNT', 'FILEDATE']
cms_df.columns
#cms_df[cols].head()

Index(['Federal Provider Number', 'Provider Name', 'Provider Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Provider Phone Number', 'Provider SSA County Code',
       'Provider County Name', 'Ownership Type', 'Number of Certified Beds',
       'Average Number of Residents per Day',
       'Average Number of Residents per Day Footnote', 'Provider Type',
       'Provider Resides in Hospital', 'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid Services',
       'Continuing Care Retirement Community', 'Special Focus Status',
       'Abuse Icon', 'Most Recent Health Inspection More Than 2 Years Ago',
       'Provider Changed Ownership in Last 12 Months',
       'With a Resident and Family Council',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Overall Rating Footnote', 'Health Inspection Rating',
       'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating Footnote',
       'Lon

In [22]:
# Analysis by Ownership
cols = ['PROVNUM',  
        'OWNERSHIP', 'BEDCERT', 'RESTOT',   'Overall_Rating','INCIDENT_CNT', 'CMPLNT_CNT', 'FINE_CNT',
        'FINE_TOT', 'PAYDEN_CNT', 'TOT_PENLTY_CNT']
        
print("=== MEAN ====")
display(cms_df.groupby(['OWNERSHIP'])[cols].mean())
print("=== SUM ====")
display(cms_df.groupby(['OWNERSHIP'])[cols].sum())
print("=== Count ====")
display(cms_df.groupby(['OWNERSHIP'])[cols].count())

=== MEAN ====


KeyError: 'OWNERSHIP'

In [ ]:
print(len(cms_df))
len(cms_df.PROVNUM.unique())


# 2013-07-01 Data

In [ ]:
cms_df_2013 = pd.read_csv('../CMS NursingHome Data/2013-07-01/ProviderInfo.csv', skipfooter=1, engine='python')
# Analysis by Ownership
# cols = ['PROVNUM',  
#         'OWNERSHIP', 'BEDCERT', 'RESTOT',   'Overall_Rating','INCIDENT_CNT', 'CMPLNT_CNT', 'FINE_CNT',
#         'FINE_TOT', 'PAYDEN_CNT', 'TOT_PENLTY_CNT']
cols=['Federal Provider Number', 
#        'Provider Name', 'Provider Address',
#        'Provider City', 'Provider State', 'Provider Zip Code',
#        'Provider SSA County Code', 'Provider County Name', 
      'Ownership Type',
       'Number of Certified Beds', 'Number of Residents in Certified Beds',
#        'Provider Type', 'Provider Resides in Hospital', 'Legal Business Name',
#        'Date First Approved to Provide Medicare and Medicaid services',
#        'Continuing Care Retirement Community', 'Special Focus Facility',
#        'Provider Changed Ownership in Last 12 Months',
#        'With a Resident and Family Council',
#        'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
#        'Overall Rating Footnote', 'Health Inspection Rating',
#        'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating Footnote',
#        'Staffing Rating', 'Staffing Rating Footnote', 'RN Staffing Rating',
#        'RN Staffing Rating Footnote', 'Reported Staffing Footnote',
#        'Physical Therapist Staffing Footnote',
#        'Reported CNA Staffing Hours per Resident per Day',
#        'Reported LPN Staffing Hours per Resident per Day',
#        'Reported RN Staffing Hours per Resident per Day',
#        'Reported Licensed Staffing Hours per Resident per Day',
#        'Reported Total Nurse Staffing Hours per Resident per Day',
#        'Reported Physical Therapist Staffing Hours per Resident Per Day',
#        'Expected CNA Staffing Hours per Resident per Day',
#        'Expected LPN Staffing Hours per Resident per Day',
#        'Expected RN Staffing Hours per Resident per Day',
#        'Expected Total Nurse Staffing Hours per Resident per Day',
#        'Adjusted CNA Staffing Hours per Resident per Day',
#        'Adjusted LPN Staffing Hours per Resident per Day',
#        'Adjusted RN Staffing Hours per Resident per Day',
#        'Adjusted Total Nurse Staffing Hours per Resident per Day',
#        'Cycle 1 Total Number of Health Deficiencies',
#        'Cycle 1 Number of Standard Health Deficiencies',
#        'Cycle 1 Number of Complaint Health Deficiencies',
#        'Cycle 1 Health Deficiency Score',
#        'Cycle 1 Standard Survey Health Date',
#        'Cycle 1 Number of Health Revisits', 'Cycle 1 Health Revisit Score',
#        'Cycle 1 Total Health Score',
#        'Cycle 2 Total Number of Health Deficiencies',
#        'Cycle 2 Number of Standard Health Deficiencies',
#        'Cycle 2 Number of Complaint Health Deficiencies',
#        'Cycle 2 Health Deficiency Score',
#        'Cycle 2 Standard Health Survey Date',
#        'Cycle 2 Number of Health Revisits', 'Cycle 2 Health Revisit Score',
#        'Cycle 2 Total Health Score',
#        'Cycle 3 Total Number of Health Deficiencies',
#        'Cycle 3 Number of Standard Health Deficiencies',
#        'Cycle 3 Number of Complaint Health Deficiencies',
#        'Cycle 3 Health Deficiency Score',
#        'Cycle 3 Standard Health Survey Date',
#        'Cycle 3 Number of Health Revisits', 'Cycle 3 Health Revisit Score',
#        'Cycle 3 Total Health Score', 'Total Weighted Health Survey Score',
       'Number of Facility Reported Incidents',
       'Number of Substantiated Complaints', 'Number of Fines',
       'Total Amount of Fines in Dollars', 'Number of Payment Denials',
       'Total Number of Penalties', 'Location', 'Processing Date']
print("=== MEAN ====")
display(cms_df_2013.groupby(['Ownership Type'])[cols].mean())
print("=== SUM ====")
display(cms_df_2013.groupby(['Ownership Type'])[cols].sum())
print("=== Count ====")
display(cms_df_2013.groupby(['Ownership Type'])[cols].count())

In [ ]:
cms_df_2013.columns

# Get Counts, Sums and Means by Ownership type and State
 - to see if any State way better/worse based on Ownership Type

In [ ]:
# Analysis by Ownership
cols = ['PROVNUM',  
        'OWNERSHIP', 'BEDCERT', 'RESTOT', 'INCIDENT_CNT', 'CMPLNT_CNT', 'FINE_CNT',
        'FINE_TOT', 'PAYDEN_CNT', 'TOT_PENLTY_CNT']
        
print("=== MEAN ====")
cms_df.groupby(['STATE', 'OWNERSHIP'])[cols].mean().to_csv('Reporting_data/CMS_Stats_2020-11-27_means.csv')
display(cms_df.groupby(['STATE', 'OWNERSHIP'])[cols].mean())
print("=== SUM ====")
cms_df.groupby(['STATE', 'OWNERSHIP'])[cols].sum().to_csv('Reporting_data/CMS_Stats_2020-11-27_sums.csv')
display(cms_df.groupby(['STATE', 'OWNERSHIP'])[cols].sum())
print("=== Count ====")
cms_df.groupby(['STATE', 'OWNERSHIP'])[cols].count().to_csv('Reporting_data/CMS_Stats_2020-11-27_counts.csv')
display(cms_df.groupby(['STATE', 'OWNERSHIP'])[cols].count())

In [ ]:
cms_df.columns

# Play Area

In [ ]:
# ToDO - Check Normalized names in spreadsheet, seems not all names Normailized ex: "ACCOLADE HEALTHCARE" and "Accolade Healthcare"
display(no_matches['Facility Names'][0:10])
display(no_matches['Facility Names Normalized'][0:10])

In [ ]:
norm_state_df.head()

In [ ]:
# Create DFs for:
#   1 - no_matches: Lists of "Factility Names" and "Facility Names Normalized" that do not have a matching CMS entry in the normalized-cms sheet
#   2 - dupNames: Dictionary of "Federal Provider Numbers" that map to list where there are more than one "Facility Name"
#   3 - CMSID2FacName: Dictionary mapping "Facility Names" to "Federal Provider Numbers"
#   4 - FacName2CMSId: Dictionary mapping "Federal Provider Numbers" to "Facility Names"

FacName2CMSId = {}
CMSID2FacName = {} # Initialize No Match to empty list so can always use extend for this key
no_matches = {'Facility Names': [],
              'Facility Names Normalized': []} # List of "Facility Names" without matches in CMS data feed
dupNames = {} # Dictionary of "Federal Provider Numbers" that have more than one "Facility Name" 
              #  (ex: 146182': ['Alden Courts of Waterford', 'Alden of Waterford'],
              #      '146186': ['Alden Estates Courts of Huntley',
              #                 'Alden Estates-Courts of Huntley'],
              #       '145872': ['Alden Long Grove Rehab & Health',
              #                  'Alden of Long Grove Rehab and HCC'], )

# For Each Federal Provder Number (CMS term) in Provide ID - CMS (CTP term)
for Fed_Prov_Num in norm_state_df['Provider ID - CMS'].unique():
    # Since the gsheets api appends comments to the cell value (which for the record can be a PAIN and POTENTIAL PROBLEM)
    #    We change the value to a more understandable "No Match"
    if Fed_Prov_Num[0:4] == "#N/A":
        no_matches['Facility Names'].extend(list(norm_state_df[norm_state_df['Provider ID - CMS'] == Fed_Prov_Num]['Facility Name'].unique()))
        no_matches['Facility Names Normalized'].extend(list(norm_state_df[norm_state_df['Provider ID - CMS'] == Fed_Prov_Num]['Facility Name Normalized'].unique()))
    else:
        CMSID2FacName[Fed_Prov_Num] = list(norm_state_df[norm_state_df['Provider ID - CMS'] == Fed_Prov_Num]['Facility Name'].unique())
        if len(CMSID2FacName[Fed_Prov_Num]) > 1:
            dupNames[Fed_Prov_Num] = CMSID2FacName[Fed_Prov_Num]
            
        for facilityName in CMSID2FacName[Fed_Prov_Num]:
            FacName2CMSId[facilityName] = Fed_Prov_Num
                              
no_matches['Facility Names'] = list(dict.fromkeys(no_matches['Facility Names']))
no_matches['Facility Names Normalized'] = list(dict.fromkeys(no_matches['Facility Names Normalized']))

In [ ]:
norm_state_df[['County', 'Facility Name', 'Provider ID - CMS']].dropna()

In [ ]:
CMSID2FacName